## Transform Awards Column

Award data comes in a string format. Format is quite consistent so a parser was constructed in order to pull out the integers.

In [3]:
import pandas as pd
import os

In [21]:
path = os.path.join('..','resources','cleaned_data', 'movies_complete_cleaned.csv')
df = pd.read_csv(path)
selected = ['name', 'awards']
df = df[selected]
df

,name,awards
0,Gold,1 win & 5 nominations.
1,The Choice,3 nominations
2,Middle School: The Worst Years of My Life,5 nominations.
3,Midnight Special,3 wins & 14 nominations.
4,A Monster Calls,39 wins & 56 nominations.
...,...,...
6550,The Karate Kid Part II,Nominated for 1 Oscar. Another 4 wins & 2 nomi...
6551,Mona Lisa,Nominated for 1 Oscar. Another 14 wins & 13 no...
6552,Ginger and Fred,Nominated for 1 Golden Globe. Another 18 wins ...
6553,Critters,3 nominations.


In [110]:
df_new = df.copy()

df_new["oscar wins and nominations"] = ''
df_new["other wins"] = ''
df_new["other nominations"] = ''
df_new["total awards"] = ''

for i, row in df.iterrows():
    a_string = row[1]
    
    oscar = 0
    wins = 0
    noms = 0
  
    try:
        splitted = a_string.split()
        ints = [int(i) for i in a_string.split() if i.isdigit()]
#         print(f"{a_string}||{len(ints)}")
        if len(ints) == 1:
            if splitted[1].strip("s.")== "nomination":
                noms = ints[0]
#                 print(f"NOM !!{a_string}")
            elif splitted[1].strip("s.")== "win":
                wins = ints[0]
#                 print(f"WIN !!{a_string}")

        elif len(ints) == 2:
            if "Another" not in splitted:
                noms = ints[1]
                wins = ints[0] 

            else:
                if "Oscar." in splitted or "Oscars." in splitted:
                    oscar = ints[0] 
                    if splitted[-1].strip("s.") == "nomination":
                        noms = ints[1] 
#                         print(f"{a_string}||{ints[0]} oscar wins+noms,{ints[1]} nomination(s)")
                    elif splitted[-1].strip("s.") == "win":
                        wins = ints[1] #other wins
#                         print(f"{a_string}||{ints[0]} oscar wins+noms, {ints[1]} wins(s)")
                else:
                    if splitted[-1].strip("s.") == "nomination":
                        noms = ints[0]+ints[1] #noms
#                         print(f"{a_string}||{ints[0]+ints[1]} nomination(s)")
                    elif splitted[-1].strip("s.") == "win":
                        noms = ints[0] #other noms
                        wins = ints[1] #wins
#                         print(f"{a_string}||{ints[0]} nomination(s) and {ints[1]} win(s)")

        elif len(ints) == 3:
            if "Oscar." in splitted or "Oscars." in splitted:
                oscar = ints[0] 
                noms = ints[2]
                wins = ints[1]  
#                 print(f"{a_string}|| {oscar} oscar wins/noms. {wins} wins. {noms} noms.")
            else:
#                 print(a_string)
                if splitted[0] == "Nominated":
                    noms = ints[0] + ints[2]
                    wins = ints[1]
                elif splitted[0] == "Won":                    
                    noms = ints[2]
                    wins = ints[0] + ints[1]
#                 print(f"{a_string}|| {wins} wins. {noms} noms.")

    except AttributeError:
        pass
        
    df_new.loc[i,"oscar wins and nominations"] = oscar
    df_new.loc[i,"other wins"] = wins
    df_new.loc[i,"other nominations"] = noms
    df_new.loc[i,"total awards"] = oscar+wins+noms

#     print(f"{a_string}|| {oscar} oscar wins or nominations. {wins} wins. {noms} nominations.")

    
df_new.head() 



,name,awards,oscar wins and nominations,other wins,other nominations,total awards
0,Gold,1 win & 5 nominations.,0,1,5,6
1,The Choice,3 nominations,0,0,3,3
2,Middle School: The Worst Years of My Life,5 nominations.,0,0,5,5
3,Midnight Special,3 wins & 14 nominations.,0,3,14,17
4,A Monster Calls,39 wins & 56 nominations.,0,39,56,95


In [116]:
path = os.path.join('..','resources','cleaned_data', 'parsed_awards_table.csv')
df_new.to_csv(path)